In [1]:
#this noetbook focus on the evaluation of summary quality using LLM
from datasets import load_dataset
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import json
# Set OpenAI's API key and API base to use vLLM's API server.
from scipy.stats import kendalltau, spearmanr
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8880/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

llm_model = "Qwen/Qwen1.5-72B-Chat"




In [2]:
#define function block

#genreate prompt
def generate_prompt(few_shot: int, aspect:str, refer_news = None, refer_summary = None, refer_score = None):
    prompt = "You are a helpful summary assistant."
    if(aspect!="Faithfulness"):
        base_prompt = "In this task, you will be provided with a news article and a generated summary.\n"\
                "Your task is to rate the " + aspect +" of the generated summary with a score from 1 to 5, "\
                "where 1 is the lowest and 5 is the highest.\n"\
                "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
        if(few_shot==0):
            prompt = base_prompt+ "Example response:\n"+aspect+" (1-5): 3"
        else:
            shot_num = len(refer_news)
            if shot_num>2:
                shot_num = 2
            if(shot_num==1):
                base_prompt+="This is an example:\n"
            else:
                base_prompt+="These are examples:\n"
            
            for i in range(shot_num):
                base_prompt = base_prompt + "News: "+refer_news[i]+"\nGenerated summary: "+refer_summary[i]+"\n"+aspect+" (1-5): "+str(refer_score[i])+"\n"
            prompt = base_prompt
    else:
        base_prompt = "In this task, you will be provided with a news article and a generated summary.\n"\
                "Your task is to rate the " + aspect +" of the generated summary with a score 0 or 1, "\
                "where 1 is not fact and 1 is fact.\n"\
                "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
        
        if(few_shot==0):
            prompt = base_prompt+ "Example response:\n"+aspect+" (0,1): 1"
        else:
            shot_num = len(refer_news)

            if shot_num>2:
                shot_num = 2
            if(shot_num==1):
                base_prompt+="This is an example:\n"
            else:
                base_prompt+="These are examples:\n"
            for i in range(shot_num):
                base_prompt = base_prompt + "News: "+refer_news[i]+"\nGenerated summary: "+refer_summary[i]+"\n"+aspect+" (0,1): "+str(refer_score[i])+"\n"
            prompt = base_prompt
                   
    return prompt
    

#get socre using openai api
def score_api_chat(client, model_name, prompt, user_input, temperature = 0):
    chat_response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": user_input},
                
            ],
            temperature=temperature
        )
        # "\n\nReference summary: "
        #         +reference_summary+
    res = chat_response.choices[0].message.content
        
    match = re.search(r':\s*(\d+)', res)   
    score = match.group(1) if match is not None else 1
    return int(score)


def loop_score_api_chat(news_list, summary_list, client, model_name, aspect, 
                        refer_news = None, refer_summary = None, refer_score = None,temperature = 0):
    
    score_list = []
    prompt = ""
    if(refer_news is None):
        prompt = generate_prompt(0, aspect)  
    else:
        prompt = generate_prompt(1, aspect, refer_news, refer_summary, refer_score)

    
    for i in range(len(news_list)):
            news = news_list[i]
            summary = summary_list[i]
            user_input = "News: "+news+"\nGenerated summary: "+summary
            score_list.append(score_api_chat(client, model_name, prompt, user_input))
    return score_list   

def correlation_score(dict1, dict2):
    #system level
    tmp_list1 = []
    tmp_list2 = []
    for i in dict1.keys():
        tmp_list1.append(np.mean(dict1[i]))
        tmp_list2.append(np.mean(dict2[i]))
    print("kendalltau correlation of system level is ", kendalltau(tmp_list1, tmp_list2)[0])
    print("spearmans correlation of system level is ", spearmanr(tmp_list1, tmp_list2)[0])
    
    #summary level
    total_corr = 0
    total_corr2 = 0
    
    for i in dict1.keys():
        total_corr+=kendalltau(dict1[i], dict2[i])[0]
        total_corr2+=spearmanr(dict1[i], dict2[i])[0]
    print("kendalltau correlation of summary level is ", total_corr/len(dict1.keys()))
    print("spearmans correlation of summary level is ", total_corr2/len(dict1.keys()))
    

In [3]:
#load the dataset to processing

cnndm_liker = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average.json')

xsum_liker = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_xsum_average.json')

summeval = pd.read_json('./filter_annotations_.jsonl')


target_dataset = cnndm_liker
template_dataset = cnndm_liker
template_index = "None"
aspect = "relevance"

tag_aspect = "relevance"


model_list = list(set(target_dataset['model'].tolist()))

model_list.remove('reference')

model_eva_dict= {}
human_eva_dict = {}
# total_summary_level = 0

# ['Relevance', 'Coherence', 'Consistency', 'Faithfulness', 'Informativeness']



template_news = [template_dataset[(template_dataset['model']=="reference" )].to_dict(orient='records')[i]['article'] for i in range(1,2)]
template_summary = [template_dataset[(template_dataset['model']=="reference" )].to_dict(orient='records')[i]['summary'] for i in range(1,2)]
template_score = [template_dataset[(template_dataset['model']=="reference" )].to_dict(orient='records')[i][aspect] for i in range(1,2)]

# template_news = [template_dataset[(template_dataset['model']=="M0" )].to_dict(orient='records')[i]['article'] for i in range(1)]
# template_summary = [template_dataset[(template_dataset['model']=="M0" )].to_dict(orient='records')[i]['summary'] for i in range(1)]
# template_score = [template_dataset[(template_dataset['model']=="M0" )].to_dict(orient='records')[i][tag_aspect] for i in range(1)]

# template_news = None
# template_summary = None
# template_score = None



for m in model_list:
    print("evaluating model: ", m)
    tmp_dataset = target_dataset[(target_dataset['model']==m )]
    
    tmp_news_list = tmp_dataset['article'].tolist()
    
    tmp_summary_list = tmp_dataset['summary'].tolist()
    tmp_score_list = tmp_dataset[tag_aspect].tolist()
    score_list = loop_score_api_chat(tmp_news_list, tmp_summary_list, client, llm_model, aspect, 
                                     refer_news = template_news, refer_summary = template_summary, refer_score = template_score)
    
    model_eva_dict[m] = score_list
    human_eva_dict[m] = tmp_score_list
    

    
#save the result
# with open('./evaluate_result/'+'qwen72_summeval_'+tag_aspect+'_template_summeval_'+str(template_index)+'_eva.json', 'w') as fp:
#     json.dump(model_eva_dict, fp)
    
# with open('./evaluate_result/'+'summeval_human_eva_dict.json', 'w') as fp:
#     json.dump(human_eva_dict, fp)

  

evaluating model:  together_glm
evaluating model:  openai_text-ada-001
evaluating model:  openai_davinci (0 shot)
evaluating model:  together_opt-175b
evaluating model:  openai_text-davinci-002
evaluating model:  anthropic_stanford-online-all-v4-s3
evaluating model:  openai_text-curie-001 (0 shot)
evaluating model:  openai_curie
evaluating model:  openai_text-curie-001
evaluating model:  pegasus
evaluating model:  cohere_xlarge-20220609
evaluating model:  openai_ada (0 shot)
evaluating model:  openai_curie (0 shot)
evaluating model:  openai_text-ada-001 (0 shot)
evaluating model:  openai_davinci
evaluating model:  openai_ada
evaluating model:  brio
evaluating model:  openai_text-davinci-002 (0 shot)


In [4]:
correlation_score(model_eva_dict, human_eva_dict)
# for i in model_eva_dict.keys():
#     print(i, ":", np.mean(model_eva_dict[i]), np.mean(human_eva_dict[i]))        

#cnndm
#index 0
# correlation of system level is  0.813119124543455
# correlation of summary level is  0.1472597601736428

#idnex 1
# correlation of system level is  0.7777777777777779
# correlation of summary level is  0.20898023319686118

#index 2
# correlation of system level is  0.33333333333333337
# correlation of summary level is  0.08772663469686875

#index 4
# correlation of system level is  0.6954252406248034
# correlation of summary level is  nan

#index None
# correlation of system level is  0.724262721160196
# correlation of summary level is  nan   

#index 0&1
# correlation of system level is  0.6997042637035166
# correlation of summary level is  0.24644284212607812



#xsum
#None
# correlation of system level is  0.7852067275033966
# correlation of summary level is  0.24612884627714107

#index 1
# correlation of system level is  0.7158720324782436
# correlation of summary level is  0.2570950849356606

#index 2
# correlation of system level is  0.2370435403783839
# correlation of summary level is  0.017176762945798965

#index 1 2 这个估计是2，然后是these are examples不同造成的结果不同
# correlation of system level is  0.7306322805705787
# correlation of summary level is  0.28464519005709527

#index 3
# correlation of system level is  0.7896732729399182
# correlation of summary level is  0.2844474928173168

# #index 2 3 
# correlation of system level is  0.8044335210322533
# correlation of summary level is  0.2894187475986224



#summeval
#None
# correlation of system level is  0.8140776760988245
# correlation of summary level is  nan
#index 1
# correlation of system level is  0.6315861769395361
# correlation of summary level is  0.32975744251357864

kendalltau correlation of system level is  0.7708704820679368
spearmans correlation of system level is  0.888203040995634
kendalltau correlation of summary level is  0.245206487822605
spearmans correlation of summary level is  0.2796228614894566


In [3]:
dict1 = json.load(open('./evaluate_result/qwen72_xsum_relevance_template_xsum_2_3_eva.json'))

dict2 = json.load(open('./evaluate_result/xsum_human_eva_dict.json'))

correlation_score(dict1, dict2)


kendalltau correlation of system level is  0.8044335210322533
spearmans correlation of system level is  0.9172288664537509
kendalltau correlation of summary level is  0.2894187475986224
spearmans correlation of summary level is  0.3382744864042585


In [4]:
#尝试最高分挑一个，最低分挑一个，然后看能不能改善结果

def get_template_list(news_list,summary_list,score_list):
    
    res_news_list = []
    res_summary_list = []
    res_score_list = []
    
    max_score = max(score_list)
    min_score = min(score_list)
    # max_index = score_list.index(max_score)
    # min_index = score_list.index(min_score)
    
    max_indexes = [index for index, value in enumerate(score_list) if value == max_score]
    min_indexes = [index for index, value in enumerate(score_list) if value == min_score]
    
    print(max_indexes, min_indexes)
    max_index = max_indexes[0]
    min_index = min_indexes[0]
    
    print(max_index, min_index)
    
    res_news_list.append(news_list[max_index])
    res_summary_list.append(summary_list[max_index])
    res_score_list.append(score_list[max_index])
    
    res_news_list.append(news_list[min_index])
    res_summary_list.append(summary_list[min_index])
    res_score_list.append(score_list[min_index])
    
    return res_news_list, res_summary_list, res_score_list


cnndm_liker = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average.json')

target_dataset = cnndm_liker
template_dataset = cnndm_liker

aspect = "relevance"

tag_aspect = "relevance"

model_list = list(set(target_dataset['model'].tolist()))
model_list.remove('reference')


candiate_news = template_dataset[template_dataset["model"]=="reference"]["article"].to_list()
candiate_summary = template_dataset[template_dataset["model"]=="reference"]["summary"].to_list()
candiate_score = template_dataset[template_dataset["model"]=="reference"][tag_aspect].to_list()

template_news, template_summary, template_score =  get_template_list(candiate_news, candiate_summary, candiate_score)



model_eva_dict= {}
human_eva_dict = {}

for m in model_list:
    print("evaluating model: ", m)
    tmp_dataset = target_dataset[(target_dataset['model']==m )]
    
    tmp_news_list = tmp_dataset['article'].tolist()
    
    tmp_summary_list = tmp_dataset['summary'].tolist()
    tmp_score_list = tmp_dataset[tag_aspect].tolist()
    score_list = loop_score_api_chat(tmp_news_list, tmp_summary_list, client, llm_model, aspect, 
                                     refer_news = template_news, refer_summary = template_summary, refer_score = template_score)
    
    model_eva_dict[m] = score_list
    human_eva_dict[m] = tmp_score_list
    

    
#save the result
with open('./evaluate_result/'+'qwen72_cnndm_'+tag_aspect+'_template_cnndm_max8_min2_eva.json', 'w') as fp:
    json.dump(model_eva_dict, fp)
    
# with open('./evaluate_result/'+'summeval_human_eva_dict.json', 'w') as fp:
#     json.dump(human_eva_dict, fp)


[8, 21, 91, 93, 97] [2]
8 2
evaluating model:  openai_davinci
evaluating model:  pegasus
evaluating model:  openai_curie (0 shot)
evaluating model:  openai_text-davinci-002
evaluating model:  openai_text-curie-001
evaluating model:  cohere_xlarge-20220609
evaluating model:  openai_text-curie-001 (0 shot)
evaluating model:  openai_text-davinci-002 (0 shot)
evaluating model:  openai_text-ada-001 (0 shot)
evaluating model:  together_opt-175b
evaluating model:  openai_davinci (0 shot)
evaluating model:  openai_curie
evaluating model:  openai_ada (0 shot)
evaluating model:  brio
evaluating model:  together_glm
evaluating model:  openai_ada
evaluating model:  anthropic_stanford-online-all-v4-s3
evaluating model:  openai_text-ada-001


In [10]:
correlation_score(model_eva_dict, human_eva_dict)

#cnndm
#none
#index None
# correlation of system level is  0.724262721160196
# correlation of summary level is  nan   


#max 8 min 2
# kendalltau correlation of system level is  0.6911252597850467
# spearmans correlation of system level is  0.8312669486241189
# kendalltau correlation of summary level is  0.2414760347942291
# spearmans correlation of summary level is  0.2754772233976758

#max 21 min 2
# kendalltau correlation of system level is  0.5940885257860047
# spearmans correlation of system level is  0.7390189760464155
# kendalltau correlation of summary level is  0.22436172250939312
# spearmans correlation of summary level is  0.2557292427393978

kendalltau correlation of system level is  0.6911252597850467
spearmans correlation of system level is  0.8312669486241189
kendalltau correlation of summary level is  0.2414760347942291
spearmans correlation of summary level is  0.2754772233976758


In [5]:
#want to evaluate without example first
#then find which is the best and worst, then use them as template

def get_template_list(candiate_news, candiate_summary, candiate_score,client, llm_model, aspect):
    res_news_list = []
    res_summary_list = []
    res_score_list = []
    
    print("evaluating without example first")
    #first evaluate
    score_list = loop_score_api_chat(candiate_news, candiate_summary, client, llm_model, aspect, 
                                     refer_news = None, refer_summary = None, refer_score = None)
    
    for i in range(len(score_list)):
        score_list[i] = abs(score_list[i]-candiate_score[i])
    
    
    max_diff_score = max(score_list)
    
    max_indexes = [index for index, value in enumerate(score_list) if value == max_diff_score]
    
    print(max_indexes)
    max_index = max_indexes[0]
    
    print(max_index)
    
    res_news_list.append(candiate_news[max_index])
    res_summary_list.append(candiate_news[max_index])
    res_score_list.append(candiate_score[max_index])
    
    
    return res_news_list, res_summary_list, res_score_list, max_index



cnndm_liker = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average.json')

target_dataset = cnndm_liker
template_dataset = cnndm_liker

aspect = "relevance"


model_list = list(set(target_dataset['model'].tolist()))
model_list.remove('reference')


candiate_news = template_dataset[template_dataset["model"]=="reference"]["article"].to_list()
candiate_summary = template_dataset[template_dataset["model"]=="reference"]["summary"].to_list()
candiate_score = template_dataset[template_dataset["model"]=="reference"][aspect].to_list()

template_news, template_summary, template_score, max_index =  get_template_list(candiate_news, candiate_summary, candiate_score, client, llm_model, aspect)



model_eva_dict= {}
human_eva_dict = {}

for m in model_list:
    print("evaluating model: ", m)
    tmp_dataset = target_dataset[(target_dataset['model']==m )]
    
    tmp_news_list = tmp_dataset['article'].tolist()
    
    tmp_summary_list = tmp_dataset['summary'].tolist()
    tmp_score_list = tmp_dataset[aspect].tolist()
    score_list = loop_score_api_chat(tmp_news_list, tmp_summary_list, client, llm_model, aspect, 
                                     refer_news = template_news, refer_summary = template_summary, refer_score = template_score)
    
    model_eva_dict[m] = score_list
    human_eva_dict[m] = tmp_score_list
    

    
#save the result
with open('./evaluate_result/'+'qwen72_cnndm_'+aspect+'_template_cnndm_diff_max'+str(max_index)+'_eva.json', 'w') as fp:
    json.dump(model_eva_dict, fp)

correlation_score(model_eva_dict, human_eva_dict)

evaluating without example first
[2]
2
evaluating model:  together_glm


KeyboardInterrupt: 